<a href="https://colab.research.google.com/github/ceabrett/tf_course/blob/master/w1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ceabrett/tf_course.git

Cloning into 'tf_course'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 30 (delta 11), reused 17 (delta 5), pack-reused 0
Unpacking objects: 100% (30/30), done.


In [5]:
# df = pd.read_csv("https://raw.githubusercontent.com/ceabrett/tf_course/master/data.csv")  
from tf_course import helper

feats_train, feats_test, label_train, label_test = helper.load_dataset_light()
# import pandas as pd
# df = pd.read_csv("tf_course/data.csv")
# df.head()  # show the beginning of the dataset
feats_train.head()

,dow_f,hour_f,log_mins_since_any_onoff,log_mins_since_utt,sun_elevation
972,6.791667,19.00,5.079546,5.080133,-0.275184
3217,2.177083,4.25,5.675765,5.645873,-0.392480
4428,0.791667,19.00,4.427650,4.428381,-0.199936
4895,5.656250,15.75,2.758081,6.249843,0.148749
5124,1.041667,1.00,5.294562,5.295060,-0.614804


In [0]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [7]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')
xs = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], dtype=float)
ys = np.array([1.0, 1.5, 2.0, 2.5, 3.0, 3.5], dtype=float)
model.fit(xs, ys, epochs=1000)
print(model.predict([7.0]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
6/6 [==============================] - 1s 89ms/sample - loss: 31.6180
Epoch 2/1000
6/6 [==============================] - 0s 555us/sample - loss: 14.6371
Epoch 3/1000
6/6 [==============================] - 0s 368us/sample - loss: 6.7778
Epoch 4/1000
6/6 [==============================] - 0s 344us/sample - loss: 3.1402
Epoch 5/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.4566
Epoch 6/1000
6/6 [==============================] - 0s 315us/sample - loss: 0.6774
Epoch 7/1000
6/6 [==============================] - 0s 333us/sample - loss: 0.3167
Epoch 8/1000
6/6 [==============================] - 0s 330us/sample - loss: 0.1497
Epoch 9/1000
6/6 [==============================] - 0s 640us/sample - loss: 0.0724
Epoch 10/1000
6/6 [==============================] - 0s 325us/sample - loss: 0.0367
Ep

In [1]:
!git clone https://github.com/ceabrett/tf_course.git

Cloning into 'tf_course'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20 (delta 5), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (20/20), done.


hello
